# loader

In [ ]:
from loader.load_data import load_fashionmnist, load_pneumoniamnist, load_chestmnist
train, test, classnames = load_fashionmnist()
# train, test, classnames = load_pneumoniamnist()
# train, test, classnames = load_chestmnist()

In [ ]:
from _utils.load_utils import display_from_batch
print(classnames)
sample_batch = next(iter(train))
display_from_batch(sample_batch, 2, 2, classnames)

# model

In [ ]:
from model.load_model import MyCNN
model = MyCNN(28, 28, 10)
numel_list = [p.numel() for p in model.parameters()]
# sum(numel_list), numel_list
print('MyCNN: ', sum(numel_list), numel_list)
from torchinfo import summary
summary(model, input_size=(1,1,28,28), device='cpu') 

# runner

In [ ]:
import datetime
current_time = datetime.datetime.today() # 2021-08-15 20:58:43.302125
current_time = current_time.strftime('%Y%m%d%H%M%S') # 20210815205827

In [ ]:
import torch
from torch import nn, optim
from runner.run_multiclass_loop import mlc_training_loop
from runner.run_multilabel_loop import mll_training_loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyCNN(28, 28, 10).to(device)
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MultiLabelSoftMarginLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
mlc_training_loop(
    n_epochs=10,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train,
    test_loader=test,
    device=device
)

In [ ]:
import collections
import tqdm
all_acc_dict = collections.OrderedDict()

# VGGnet

In [ ]:
from loader.load_data import load_fashionmnist, load_pneumoniamnist, load_chestmnist
from torchvision import transforms
trformers=transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])
train, test, classnames = load_fashionmnist(batch_size=4, trformers=trformers)
# train, test, classnames = load_pneumoniamnist()
# train, test, classnames = load_chestmnist()

from _utils.load_utils import display_from_batch
print(classnames)
sample_batch = next(iter(train))
print(sample_batch[0].shape)
display_from_batch(sample_batch, 2, 2, classnames)

In [ ]:
from model.load_model import VGG
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }
model = VGG(cfgs["E"], batch_norm=False, num_classes=10)

In [ ]:
numel_list = [p.numel() for p in model.parameters()]
# sum(numel_list), numel_list
print('VGG: ', sum(numel_list), numel_list)
from torchinfo import summary
summary(model, input_size=(2,1,224,224), device='cpu') 

In [ ]:
import torch
x = torch.randn(4,1,224,224)
print(model(x).shape)
print(model(x))
_, y_pred = model(x).max(1)
print(y_pred)

In [ ]:
import torch
from torch import nn, optim
from runner.run_multiclass_loop import mlc_training_loop
from runner.run_multilabel_loop import mll_training_loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MultiLabelSoftMarginLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
mlc_training_loop(
    n_epochs=4,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train,
    test_loader=test,
    device=device
)

# AlexNet

In [ ]:
from loader.load_data import load_fashionmnist, load_pneumoniamnist, load_chestmnist, load_cifar10
train, test, classnames = load_cifar10()
# train, test, classnames = load_fashionmnist()
# train, test, classnames = load_pneumoniamnist()
# train, test, classnames = load_chestmnist()

In [ ]:
from _utils.load_utils import display_from_batch
print(classnames)
sample_batch = next(iter(train))
print(sample_batch[0].shape)
display_from_batch(sample_batch, 2, 2, classnames)

In [ ]:
from model.load_model import MyCNN, AlexNet
model = AlexNet(3, 32, 32, 10)
numel_list = [p.numel() for p in model.parameters()]
# sum(numel_list), numel_list
print('AlexNet: ', sum(numel_list), numel_list)
from torchinfo import summary
summary(model, input_size=(1,3,32,32), device='cpu') 

In [ ]:
import torch
x = torch.randn(4,3,32,32)
print(model(x).shape)
print(model(x))
_, y_pred = model(x).max(1)
print(y_pred)

In [ ]:
import torch
from torch import nn, optim
from runner.run_multiclass_loop import mlc_training_loop
from runner.run_multilabel_loop import mll_training_loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=1e-2)
train_losses, train_acc, val_acc = mlc_training_loop(
    n_epochs=20,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train,
    test_loader=test,
    device=device
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()